## Application - Face detection

References:

- https://towardsdatascience.com/face-detection-for-beginners-e58e8f21aad9
- https://towardsdatascience.com/cnn-based-face-detector-from-dlib-c3696195e01c

In [ ]:
from PIL import Image
import numpy as np
import os
import sys
import time
import cv2
import dlib
import requests
from io import BytesIO

In [ ]:
cascade = cv2.CascadeClassifier('./face/haarcascade_frontalface_default.xml')
eyeCascade = cv2.CascadeClassifier('./face/haarcascade_eye.xml')

def detectFaceCV(gray):

    start_time = time.time()
    faces = []
    try:
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #gray = cv2.equalizeHist(gray)
        rects = cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=4, minSize=(30, 30),flags=cv2.CASCADE_SCALE_IMAGE)
        
        for rect in rects:
            (x, y, w, h) = rect
            roi = gray[y:y+h, x:x+w]
            faces.append(rect)

    except Exception as e:
        print(e)

    return faces, time.time() - start_time

In [ ]:
def detectFaceEyeCV(gray):

    start_time = time.time()
    faces = []
    try:
        rects = cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=4, minSize=(30, 30),flags=cv2.CASCADE_SCALE_IMAGE)
        
        for rect in rects:
            (x, y, w, h) = rect
            roi = gray[y:y+h, x:x+w]
            eyes = eyeCascade.detectMultiScale(roi)
            if len(eyes):
                faces.append(rect)

    except Exception as e:
        print(e)

    return faces, time.time() - start_time

In [ ]:
hog_face_detector = dlib.get_frontal_face_detector()

def detectFaceHog(gray):
    
    start_time = time.time()   
    rects = []
    try:
        rects = hog_face_detector(gray, 1)
        
        faces = [ [rect.left(), rect.top(), rect.right()-rect.left(), rect.bottom()-rect.top()] for rect in rects ]

    except Exception as e:
        print(e)

    return faces, time.time() - start_time

In [ ]:
#WARNING: too slow to detect in realtime with CPU
cnn_face_detector = dlib.cnn_face_detection_model_v1("./face/mmod_human_face_detector.dat")

def detectFaceCNN(gray):
    
    start_time = time.time()
    rects = []
    try:
        rects = cnn_face_detector(gray, 1)
        
        faces = [ [rect.rect.left(), rect.rect.top(), rect.rect.right()-rect.rect.left(), rect.rect.bottom()-rect.rect.top()] for rect in rects ]

    except Exception as e:
        print(e)

    return faces, time.time() - start_time

### Test

In [ ]:
# resize size of image width
_resize = 640 #1000

In [ ]:
# detect with reduced size
def displayface(img, detectfn=detectFaceCV):
       
    # resize image
    imgh, imgw, imgc = img.shape
    scale = 1.0
    if (imgh > _resize  or imgw > _resize ):
        scale = imgh if (imgh > imgw) else imgw
        scale = _resize / scale

    img = cv2.resize(img, (int(imgw*scale), int(imgh*scale)), interpolation = cv2.INTER_AREA)
    
    # detect
    faces, _ = detectfn(img)

    numface = len(faces)
    for face in faces:
        print(face)
        (x, y, w, h) = face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
    imgdetect = Image.fromarray(img)
    display(imgdetect)

In [ ]:
# compare each detection algorithm
def detectcompare(img):
       
    # resize image
    imgh, imgw, imgc = img.shape
    scale = 1.0
    if (imgh > _resize  or imgw > _resize ):
        scale = imgh if (imgh > imgw) else imgw
        scale = _resize / scale

    img = cv2.resize(img, (int(imgw*scale), int(imgh*scale)), interpolation = cv2.INTER_AREA)
    
    # detect
    faces1, t1 = detectFaceCV(img)
    faces2, t2 = detectFaceHog(img)
    faces3, t3 = detectFaceCNN(img)

    for face in faces1:
        (x, y, w, h) = face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)


    for face in faces2:
        (x, y, w, h) = face
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
    for face in faces3:
        (x, y, w, h) = face
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
    imgdetect = Image.fromarray(img)
    display(imgdetect)
    
    print("cv(red): {:.2f} sec, hog(green): {:.2f} sec, cnn(blue): {:.2f} sec".format(t1, t2, t3))

In [ ]:
#test samples
test_urls = [ 'http://www.science-alive.co.uk/wp-content/uploads/2012/04/Young-people.jpg' ]

In [ ]:
for test_url in test_urls:
    r = requests.get(test_url)
    img = Image.open(BytesIO(r.content))
    img = np.array(img)
    
    displayface(img, detectFaceCV)

In [ ]:
for test_url in test_urls:
    r = requests.get(test_url)
    img = Image.open(BytesIO(r.content))
    img = np.array(img)
    
    detectcompare(img)

### Live Detection (Camera capture)

__Note__: run this on local PC

In [ ]:
cap = cv2.VideoCapture(0)
print('width: {}, height: {}'.format(cap.get(3),cap.get(4)))
cap.set(3,320)
cap.set(4,240)

ts, f, i = time.time(), 0, 0;
x, y, w, h = 0, 0, 0, 0
dt = time.time()
while(True):

    ret, frame = cap.read()

    if (ret):

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects, _ = detectFaceEyeCV(gray)
        #rects, _ = detectFaceHog(gray)
               
        for rect in rects:
            (x, y, w, h) = rect
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        ### keep preview detection region - for multiple faces?
        #dt = time.time() if len(rects) else dt
        #if (time.time() - dt < 1.0):
        #    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
        te = time.time()
        i += 1
        if (te - ts > 1.0):
            ts = te
            f = i
            i = 0
            
        cv2.putText(frame, "{}fps".format(f), (10,40), 2, 0.7, (255, 0, 0))        
        cv2.imshow('frame', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()